In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import pickle

pd.set_option("display.max_columns", None)

def run_report(start, end):
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)

    market_cap = pd.read_csv('input/market_cap.csv')

    # Melt the DataFrame to have a long format with Date, Ticker, and Market Cap
    firm_market_caps = market_cap.melt(id_vars=['Date'], var_name='Ticker', value_name='firm_date_market_cap')
    firm_market_caps['Date'] = pd.to_datetime(firm_market_caps['Date'])

    # Read the ray data
    ray_df = pd.read_excel('input/RAY as of OCT 23 20231_PA.xlsx')
    ray_df = ray_df.sort_values('Market Cap\n', ascending=False)

    # Clean the columns and ticker values
    ray_df.columns = ray_df.columns.str.rstrip('\n')
    ray_df['Ticker'] = ray_df['Ticker'].str.split(' ', n=1, expand=True)[0].replace(' ', '')

    # Filter relevant columns and rows
    ray_df = ray_df[['Ticker', 'GICS Sector']].head(100)

    # Merge the DataFrames
    report_df = pd.merge(firm_market_caps, ray_df, on='Ticker')

    # Filter by date range and sort the DataFrame
    report_df = report_df[(report_df['Date'] >= start_date) & (report_df['Date'] <= end_date)]
    report_df = report_df.sort_values('Date')

    # Group by Date and GICS Sector and sum the market caps
    report_df = report_df.groupby(['Date', 'GICS Sector'])['firm_date_market_cap'].sum().reset_index()
    report_df.columns = ['Date', 'GICS Sector', 'Market Cap']

    # Calculate the returns
    report_df['Return'] = report_df.groupby('GICS Sector')['Market Cap'].transform(lambda x: ((x / x.iloc[0]) - 1) * 100)

    # Calculate the total market cap per date
    total_market_cap = report_df.groupby('Date')['Market Cap'].sum().reset_index()
    total_market_cap.columns = ['Date', 'Total Market Cap']

    # Merge the total market cap with the report DataFrame
    report_df = pd.merge(report_df, total_market_cap, on='Date')

    run_report.report = report_df

    # Calculate the index return
    run_report.index_return = (((report_df.iloc[-1]['Total Market Cap'] / report_df.iloc[0]['Total Market Cap']) - 1) * 100)














    input_file_path = os.path.join(os.getcwd(), "input/date_dataframes.pkl")

    with open(input_file_path, 'rb') as input_file:
        date_dataframes = pickle.load(input_file)

    df = pd.read_excel('input/RAY as of OCT 23 20231_PA.xlsx')
    df = df.sort_values('Market Cap\n',ascending=False)  
    df.columns = df.columns.str.rstrip('\n')
    df['Ticker'] = df['Ticker'].str.split(' ',n=1,expand=True)[0].replace(' ','')
    float_df = df[['Ticker','Equity Float']].head(100)
    float_df

    eod_market_cap_pivot_close = pd.DataFrame()
    eod_market_cap_pivot_mktcap = pd.DataFrame()

    for date in list(date_dataframes.keys()):
        if date >= start and date <= end:
            eod_df = date_dataframes[date]

            merged_df = pd.merge(float_df,eod_df,left_on='Ticker',right_on='underlying_symbol')

            merged_df['Market Cap'] = merged_df['Equity Float'] * merged_df['close']
    
            eod_market_cap_daily_close = merged_df.pivot_table(values='close',index='quote_date',columns='Ticker',aggfunc='sum')

            eod_market_cap_daily_mktcap = merged_df.pivot_table(values='Market Cap',index='quote_date',columns='Ticker',aggfunc='sum')
    
            eod_market_cap_pivot_close = pd.concat([eod_market_cap_pivot_close,eod_market_cap_daily_close])

            eod_market_cap_pivot_mktcap = pd.concat([eod_market_cap_pivot_mktcap,eod_market_cap_daily_mktcap])

    eod_market_cap_pivot_close = eod_market_cap_pivot_close.rename_axis(index='Date')
    eod_market_cap_pivot_close.index = pd.to_datetime(eod_market_cap_pivot_close.index,errors='coerce')

    eod_market_cap_pivot_mktcap = eod_market_cap_pivot_mktcap.rename_axis(index='Date')
    eod_market_cap_pivot_mktcap.index = pd.to_datetime(eod_market_cap_pivot_mktcap.index,errors='coerce')


    run_report.eod_market_cap_pivot_close = eod_market_cap_pivot_close
    run_report.eod_market_cap_pivot_mktcap = eod_market_cap_pivot_mktcap







    from collections import defaultdict

    # Initialize a dictionary to hold lists of tickers for each GICS sector
    gics_sector_dict = defaultdict(list)

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        gics_sector = row['GICS Sector']
        ticker = row['Ticker']
        gics_sector_dict[gics_sector].append(ticker)

    # Convert defaultdict to a regular dict (optional)
    gics_sector_dict = dict(gics_sector_dict)






    total_market_cappp = eod_market_cap_pivot_mktcap.sum(axis=1)[0]

    stock_price_data = {}
    sector_weight = {}

    for sector in gics_sector_dict.keys():
        info = []
        marketcap = 0
        for ticker in eod_market_cap_pivot_close.keys():
            if ticker in gics_sector_dict[sector]:
                ret = ((((eod_market_cap_pivot_close.at[eod_market_cap_pivot_close.index[-1],ticker]) / (eod_market_cap_pivot_close.at[eod_market_cap_pivot_close.index[0],ticker])) -1 ) * 100)
                mktcap = eod_market_cap_pivot_mktcap.at[eod_market_cap_pivot_mktcap.index[0],ticker]
                info.append({'Ticker': ticker, 'Return': ret, 'MktCap': mktcap})

                marketcap += mktcap

        stock_price = pd.DataFrame(info)

        total_weight = stock_price['MktCap'].sum()
        stock_price['LocalWeight'] = round((stock_price['MktCap'] / total_weight),6)*100
        stock_price['Weight'] = round((stock_price['MktCap'] / report_df['Total Market Cap'].head(1).values[0]),6)*100
        stock_price['RelativeReturn'] = round((stock_price['Return'] * (stock_price['LocalWeight'] / 100)),6)

        stock_price_data[sector] = stock_price

        weight = marketcap / total_market_cappp

        sector_weight[sector] = weight


    sec = pd.DataFrame(sector_weight,index=sector_weight.values()).iloc[0].to_frame().reset_index()
    sec.columns = ['GICS Sector','Weight']
    sec = sec.sort_values('Weight',ascending=False).reset_index().drop(columns='index')
    run_report.sector_weight = sec
    run_report.stock_price_data = stock_price_data

    run_report.final_day_values = report_df.iloc[-11:]














    max_return = report_df.iloc[-11:]['Return'].max()
    min_return = report_df.iloc[-11:]['Return'].min()


    for i,ret in enumerate(report_df.iloc[-11:]['Return']):
        if ret == max_return:
            max_gics_sector = report_df.iloc[i]['GICS Sector']
            max = stock_price_data[max_gics_sector]
        if ret == min_return:
            min_gics_sector = report_df.iloc[i]['GICS Sector']
            min = stock_price_data[min_gics_sector]

    max_rel_return = max['RelativeReturn'].max()
    min_rel_return = min['RelativeReturn'].min()

    for i,rel_ret in enumerate(max['RelativeReturn']):
        if rel_ret == max_rel_return:
            max_ticker = max.iloc[i]['Ticker']
            max_ticker_return = max.iloc[i]['Return']
            max_ticker_localweight = max.iloc[i]['LocalWeight']
            max_ticker_weight = max.iloc[i]['Weight']

    for i,rel_ret in enumerate(min['RelativeReturn']):
        if rel_ret == min_rel_return:
            min_ticker = min.iloc[i]['Ticker']
            min_ticker_return = min.iloc[i]['Return']
            min_ticker_localweight = min.iloc[i]['LocalWeight']
            min_ticker_weight = min.iloc[i]['Weight']

    statement = (f"The highest performing sector during this period was the {max_gics_sector} sector, with a return of {round(max_return,2)}%. This was primarily led by {max_ticker}\
, whose stock returned {round(max_ticker_return,2)}%. {max_ticker} makes up {round(max_ticker_localweight,2)}% of the {max_gics_sector} sector and {round(max_ticker_weight,2)}% of the entire index. \
The lowest performing sector during this period was the {min_gics_sector} sector, with a return of {round(min_return,2)}%. This was primarily led by {min_ticker}\
, whose stock returned {round(min_ticker_return,2)}%. {min_ticker} makes up {round(min_ticker_localweight,2)}% of the {min_gics_sector} sector and {round(min_ticker_weight,2)}% of the entire index.")

    run_report.statement = statement
    run_report.best_sector = max
    run_report.worst_sector = min















    
    def make_returns_over_time(report):
        def create_returns_over_time():
            plt.figure(figsize=(12, 8))

            # Loop through each sector and plot the data
            for sector in report['GICS Sector'].unique():
                sector_data = report[report['GICS Sector'] == sector]
                plt.plot(sector_data['Date'], sector_data['Return'], label=sector)

            # Set major formatter for dates to display only date part
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

            # Set major locator to display only unique dates
            unique_dates = report['Date'].unique()
            plt.gca().set_xticks(unique_dates)

            # Add labels, title, and legend
            plt.xlabel('Date')
            plt.ylabel('Return')
            plt.title('Returns Over Time by Sector')
            plt.legend(title='GICS Sector')

            # Rotate x-axis labels for better readability
            plt.xticks(rotation=45, ha='right')

            # Adjust layout for better fit
            plt.tight_layout()

            # Show the plot
            plt.show()

        return create_returns_over_time

    ret_over_time = make_returns_over_time(report_df)








    def make_pie_chart(sector_weight):
        def create_pie_chart():
            # Create the pie chart
            labels = list(sector_weight.keys())
            sizes = list(sector_weight.values())

            # Define a new color palette
            pastel_colors = [
            '#FFB3BA',
            '#FFDFBA',
            '#FFFFBA',
            '#BAFFC9',
            '#BAE1FF',
            '#CBAACB',
            '#FFDAC1',
            '#B5EAD7',
            '#E2F0CB',
            '#C6DBDA',
            '#F3C4FB'
            ]

            # Custom function to format labels
            def autopct_format(values):
                def my_format(pct):
                    return ('%.1f%%' % pct) if pct > 4 else ''
                return my_format

            fig, ax = plt.subplots()
            ax.pie(sizes, labels=labels, autopct=autopct_format(sizes), startangle=140, colors=pastel_colors)
            ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

            # Display the pie chart
            plt.title("Sector Weights")
            plt.show()
            plt.close()
    
        return create_pie_chart

    # Assign the function to a variable
    seccy_weight = make_pie_chart(sector_weight)

    # Call the variable to display the pie chart



    def make_bar_chart(dataframe):
        def create_bar_chart():


            df = dataframe.iloc[-11:][['GICS Sector','Return']]

            # Sort the DataFrame by 'GICS Sector' in descending alphabetical order
            df = df.sort_values(by='GICS Sector', ascending=False)

            # Plotting the bar chart
            plt.figure(figsize=(10, 6))
            colors = ['green' if x > 0 else 'red' for x in df['Return']]
            bars = plt.barh(df['GICS Sector'], df['Return'], color=colors)

            # Adding values to the ends of the bars
            for bar in bars:
                width = bar.get_width()
                plt.text(width + 0.10 if width > 0 else width - 0.10, 
                         bar.get_y() + bar.get_height() / 2, 
                         f'{width:.2f}%', 
                         ha='left' if width > 0 else 'right', 
                         va='center', 
                         color='black')

            # Setting x-axis limits
            plt.xlim(-6, 4)

            # Removing the border
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)
            plt.gca().spines['left'].set_visible(False)
            plt.gca().spines['bottom'].set_visible(False)

            # Labels and title
            plt.xlabel('Return (%)')
            plt.ylabel('GICS Sector')
            plt.title('Index Return')
            plt.grid(axis='x', linestyle='--', alpha=0.7)

            # Adjust the margins to make space for the labels
            plt.subplots_adjust(left=0.25)

            plt.show()
            plt.close()
        return create_bar_chart
    thebarchart = make_bar_chart(report_df)



    

    


    
    return ret_over_time(),seccy_weight(),thebarchart()

In [ ]:
run_report('2024-06-01','2024-06-30')

In [ ]:
run_report.index_return

In [ ]:
# run_report.report

In [ ]:
run_report.final_day_values

In [ ]:
run_report.eod_market_cap_pivot_close

In [ ]:
run_report.eod_market_cap_pivot_mktcap

In [ ]:
run_report.sector_weight.sort_values('GICS Sector')

In [ ]:
# run_report.stock_price_data

In [ ]:
run_report.best_sector

In [ ]:
run_report.worst_sector

In [ ]:
run_report.statement